In [1]:
Dataset_size = 10000
g_star = 'both'       # low or high or both
model = 'WS'     # FC or WS
g_known = False   # True or False
BATCH_SIZE = 100
C = 1
Epochs = 500

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from keras.datasets import mnist
import pickle
import random
import math
import tensorflow as tf
import os



class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
    
class Model_h(nn.Module):
    def __init__(self):
        super(Model_h, self).__init__()
        
        if(model == 'FC') :
            self.W = nn.Parameter(torch.tensor(np.zeros((10,10)), dtype = torch.float, requires_grad = True))
        elif(model == 'WS') :
            self.W = nn.Parameter(torch.tensor(np.zeros((1,10)), dtype = torch.float, requires_grad = True))
            
        torch.nn.init.xavier_uniform_(self.W)
        
    def forward(self, inputs):
        
        x = inputs * self.W
        x = torch.tanh(x)
        x = x.sum(dim = -1)
        
        return x

class Model_g(nn.Module) :
    def __init__(self, C=1):
        super(Model_g, self).__init__()
        
        self.C = 1

        if( not g_known) : 
            self.layer_list = nn.ModuleList()
            self.layer_list.append(nn.Linear(10, 50))
            self.layer_list.append(nn.Linear(50, 1))
        
    def forward(self, inputs):
        
        if(g_known) :
            if(g_star=='low') :
                return inputs[:, 0]
            elif(g_star=='high') :
                return torch.cos(inputs[:, :5].sum(-1))
            elif(g_star=='both') :
                return inputs[:, 0]*self.C+  torch.cos(inputs[:, :5].sum(-1))
        else:
            x = self.layer_list[0](inputs)
            x = torch.relu(x)
            x = self.layer_list[1](x)
            return x[:,0]

#for d_set in ["gaussian", "mnist"] :
for d_set in [ "mnist"] :

  if(d_set == "gaussian") :
    temp =[True, False]
  else :
    temp =[False]

  for g_known in temp :

    for model in ['WS', 'FC'] :
    #for model in ['FC'] :

      #for g_star in ["high", "low", "both"] :
      for g_star in ["low"] :

        if (g_star == "both") :
          #temp2 = [2, 5]
          temp2= [0.5]
        else :
          temp2 = [1]

        for C in temp2 :

          print(d_set+"_" +str(C)+"_"+str(g_known)+"_"+model+"_"+g_star)

          np.random.seed(0)

          if (d_set == "gaussian") :
            X = np.random.normal(size = (Dataset_size, 10, 10))
          else :
            (trainX, trainy), (testX, testy) = mnist.load_data()
            trainX = np.expand_dims(trainX, 3)
            trainX.shape

            X = tf.image.extract_patches(trainX, (1, 10, 10, 1), (1, 5, 5, 1), (1,1,1,1), padding='VALID')
            X = np.reshape(X, (-1, 100))
            X = np.reshape(X, (X.shape[0], 10, 10))
            X = X[np.random.choice(range(X.shape[0]), 10000)]

          U_0 = np.random.normal(size = (10,1))

          X_temp = np.sum(np.tanh(X * U_0.T), axis = -1)

          if(g_star == 'low') :
              Y = X_temp[:, 0]
          if(g_star == 'high') :
              Y = np.cos(np.sum(X_temp[:, :5], axis = -1))
          if(g_star == 'both') :
              Y = X_temp[:, 0]*C + np.cos(np.sum(X_temp[:, :5], axis = -1))

          g = Model_g(C)
          h = Model_h()

          train_data = trainData(torch.FloatTensor(X), torch.FloatTensor(Y))
          train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
          criteria = nn.MSELoss()
          optimizer = optim.SGD(list(g.parameters())+list(h.parameters()), lr = 0.01)

          g.train()
          h.train()

          res = np.zeros(Epochs)

          for epoch in range(Epochs) :
              epoch_loss = 0
              for X_batch, y_batch in train_loader :
                  optimizer.zero_grad()
                  y_pred = g(h(X_batch))
                  loss = criteria(y_pred, y_batch)
                  loss.backward()
                  optimizer.step()
                  epoch_loss += loss.item()/2
              if(epoch % 50 == 0):
                  print(epoch_loss/len(train_loader))
              res[epoch] = epoch_loss/len(train_loader)

          file = open(d_set+"_" +str(C)+"_"+str(g_known)+"_"+model+"_"+g_star, 'wb')
          pickle.dump(res, file)
          file.close()

mnist_1_False_WS_low
0.6188299231231212
0.0015229637344600633
0.0010277310939272866
0.0008231849098228849
0.0006666125651099719
0.0005177800044475589
0.00036486933342530395
0.00028293547533394305
0.0002517066165819415
0.00023797589703463017
mnist_1_False_FC_low
0.8835350194573403
0.24062709808349608
0.2024803874641657
0.09175483401864767
0.08775918759405613
0.0849130305275321
0.08239796828478575
0.08157896568998695
0.0798985019698739
0.07897066796198488


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
g_star

'both'